In [ ]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import json
import pandas as pd
import sqlite3
from geopy.geocoders import Nominatim

In [ ]:
# Set executable path and initialize Chrome browser

# MAC
# executable_path = {"executable_path": "/usr/local/bin/chromedriver"}

# Windows
executable_path = {"executable_path": "chromedriver.exe"}

browser = Browser("chrome", **executable_path, headless=True)

In [ ]:
# Visit the backpacker website
url = "https://www.priceoftravel.com/world-cities-by-price-backpacker-index/"
browser.visit(url)

time.sleep(1)

# Find all cities in top list
top_cities_html = browser.html
top_cities_soup = bs(top_cities_html, "html.parser")

top_cities = top_cities_soup.find('div', class_ = "bpiidx_list").find_all('a', href=True)
daily_total = top_cities_soup.find('div', class_ = "bpiidx_list").find_all('div', class_="bpidx price")

# create a dict for each set of information type
city_facts_dict = {}
transport_dict = {}
food_drinks_dict = {}
temp_prcp_dict = {}
hotel_dict = {}
hostel_dict = {}
intro_dict = {}

# create all the lists that we need to store the information
places_list = []
rank_list = []
daily_total_list = []
population_list = []
metro_list = []
timezone_list = []
currency_list = []
airport_list = []
transport_city_list = []
transport_mode_list = []
transport_upper_price_list = []
transport_lower_price_list = []
food_city_list = []
food_type_list = []
food_desc_list = []
food_upper_price_list = []
food_lower_price_list = []
temp_prcp_cityList = []
month_list = []
high_temp_list = []
low_temp_list = []
prcp_inch_list = []
hotel_cityList = []
hotel_ratingsList = []
hotel_upper_priceList = []
hotel_lower_priceList = []
hostel_cityList = []
hostel_upper_priceList = []
hostel_lower_priceList = []
intro_p1 = []
intro_p2 = []
intro_p3 = []
intro_p4 = []

In [ ]:
for city in range(0,len(top_cities)):
        
    city_country_name = top_cities[city].text.replace("\n"," ").strip()
    places_list.append(city_country_name)
    
    # get daily total
    city_daily_total = daily_total[city]
    city_daily_total = city_daily_total.text.replace("\n"," ").strip()
    city_daily_total = city_daily_total.split("$")
    city_daily_total = city_daily_total[1]
    daily_total_list.append(city_daily_total)

    # get the links to each of the top cities page
    top_city = top_cities[city]
    top_cities_link = top_city.get_attribute_list('href')[0]
    browser.visit(top_cities_link)

    #######################
    # Scrape facts table
    #######################
    try:
        # getting the city facts
        browser_html = browser.html
        browser_soup = bs(browser_html, "html.parser")
        
        population = browser_soup.find("table", class_ = "col city-facts").find_all('td', class_ = "white")[0]
        population = population.text.replace("\n"," ").strip()
        population = float(population.replace(',',''))

        metro = browser_soup.find("table", class_ = "col city-facts").find_all('td', class_ = "white")[1]
        metro = metro.text.replace("\n"," ").strip()
        metro = float(metro.replace(',',''))

        timezone = browser_soup.find("table", class_ = "col city-facts").find_all('td', class_ = "white")[2]
        timezone = timezone.text.replace("\n"," ").strip()

        currency = browser_soup.find("table", class_ = "col city-facts").find_all('td', class_ = "white")[3]
        currency = currency.text.replace("\n"," ").strip()

        airport = browser_soup.find("table", class_ = "col city-facts").find_all('td', class_ = "white")[4]
        airport = airport.text.replace("\n"," ").strip()
          
    except AttributeError:
        population = ""
        metro = ""
        timezone = ""
        currency = ""
        airport = ""

    # append data to lists
    rank_list.append(city+1)
    population_list.append(population)
    metro_list.append(metro)
    timezone_list.append(timezone)
    currency_list.append(currency)
    airport_list.append(airport)
    
    print(f"{city+1}")
    
    ########################
    # Scrape intro paragraph
    ########################
    try:
        # getting the city facts
        browser_html = browser.html
        browser_soup = bs(browser_html, "html.parser")
        
        intro1 = browser_soup.find(id="content").find_all('p')[0]
        intro1 = intro1.text.replace("\n"," ").strip()
        
        intro2 = browser_soup.find(id="content").find_all('p')[1]
        intro2 = intro2.text.replace("\n"," ").strip()
        
        intro3 = browser_soup.find(id="content").find_all('p')[2]
        intro3 = intro3.text.replace("\n"," ").strip()
        
        intro4 = browser_soup.find(id="content").find_all('p')[3]
        intro4 = intro4.text.replace("\n"," ").strip()
        
    except:
        # getting the city facts
        browser_html = browser.html
        browser_soup = bs(browser_html, "html.parser")
        
        intro1 = browser_soup.find(id="content").find_all('p')[0]
        intro1 = intro1.text.replace("\n"," ").strip()
        
        intro2 = browser_soup.find(id="content").find_all('p')[1]
        intro2 = intro2.text.replace("\n"," ").strip()
        
        intro3 = browser_soup.find(id="content").find_all('p')[2]
        intro3 = intro3.text.replace("\n"," ").strip()
        
        intro4 = "empty"
        
    # append data to lists
    intro_p1.append(intro1)
    intro_p2.append(intro2)
    intro_p3.append(intro3)
    intro_p4.append(intro4)
    
    print(f"#{city+1}: {city_country_name}; {intro4}")
    
    ################################
    # Scrape transportation prices
    ################################

    try:
        # Gather transportation information
        
        browser_html = browser.html
        browser_soup = bs(browser_html, "html.parser")
        
        transport_table = browser_soup.find("table", class_ = "fcol fcol-padding").find_all('tr', class_ = "border no-pad")
        
        for transport in range(0,len(transport_table)):
            
            # get transport mode/type
            transport_mode = transport_table[transport].find('td', class_="white")
            transport_mode = transport_mode.text.replace("\n"," ").strip()
            transport_mode_list.append(transport_mode)

            # get transportation price
            transport_price = transport_table[transport].find('td', class_="white2")
            transport_price = transport_price.text.replace("\n"," ").strip()
            
            # split the price range into upper and lower range
            try:
                transport_price_range = transport_price.split(" - ")
                lower_price = transport_price_range[0]
                upper_price = transport_price_range[1]
            except:
                lower_price = transport_price
                upper_price = transport_price
                
            transport_lower_price_list.append(lower_price)
            transport_upper_price_list.append(upper_price)
            
            # get country list
            transport_city_list.append(city_country_name)
                                           
    except AttributeError:
        pass
    
    ################################
    # Scrape food and drink prices
    ################################
    
    # Gather food information
    try:
        browser_html = browser.html
        browser_soup = bs(browser_html, "html.parser")
        
        food_table = browser_soup.find_all("table", class_ = "fcol fcol-padding")[1]
        food_table = food_table.find_all('td', class_ = "white")
        
        for food in range(0,len(food_table),3):  

            # get food type
            food_type = food_table[food].text.replace("\n"," ").strip()
            food_type_list.append(food_type)

            # get description
            food_desc = food_table[food+1].text.replace("\n"," ").strip()
            food_desc_list.append(food_desc)

            # get food price
            food_price = food_table[food+2].text.replace("\n"," ").strip()
            
            # split the range and put into lower and upper price range
            try:
                food_price_range = food_price.split(" - ")
                lower_price = food_price_range[0]
                upper_price = food_price_range[1]
            except:
                lower_price = food_price
                upper_price = food_price
                
            food_lower_price_list.append(lower_price)
            food_upper_price_list.append(upper_price)
            
            # get country list
            food_city_list.append(city_country_name)

    except IndexError:
        pass
        
    ########################################
    # Scrape temperature & precipitation
    ########################################
    
    try:
        browser_html = browser.html
        browser_soup = bs(browser_html, "html.parser")
        
        table = browser_soup.find("table", class_ = "center-table weather-tab").find_all("td")

        # Gather temperature and precipitation information
        for data in range(0,len(table),4):

            # get month
            month = table[data].text.replace("\n"," ").strip()
            month_list.append(month)

            # get high temp
            hi_temp = table[data+1].text.replace("\n"," ").strip()
            high_temp_list.append(hi_temp)

            # get low temp
            lo_temp = table[data+2].text.replace("\n"," ").strip()
            low_temp_list.append(lo_temp)

            # get precipitation in inches
            prcp = table[data+3].text.replace("\n"," ").strip()
            prcp_inch_list.append(prcp)
            
            # get country list
            temp_prcp_cityList.append(city_country_name)
            
    except AttributeError:
        pass
    
    ########################################
    # Scrape hotel and hostel information
    ########################################
    
    try:
        browser_html = browser.html
        browser_soup = bs(browser_html, "html.parser")
        
        #################
        # Hotel
        #################
        
        hotel_table = browser_soup.find("table", class_ = "ratings").find_all('td')
                
        # get hotel ratings
        for i in range(1,6):
            if i == 1:
                stars = f"{i}_star"
            else:
                stars = f"{i}_stars"
            hotel_ratingsList.append(stars)        
        
        for data in range(1,len(hotel_table),2):
        
            # get country list
            hotel_cityList.append(city_country_name)
            
            # get hotel price        

            hotel_price = hotel_table[data].text.replace("\n"," ").strip()
            
            try:
                hotel_price_list = hotel_price.split(" - ")
                hotel_lower_price = hotel_price_list[0]
                hotel_upper_price = hotel_price_list[1]
            except:
                hotel_lower_price = hotel_price
                hotel_upper_price = hotel_price
                
            hotel_lower_priceList.append(hotel_lower_price)
            hotel_upper_priceList.append(hotel_upper_price)
                                                  
            # print to check scraping status
            # print(f"#{city+1}: {city_country_name}; {stars}: {hotel_lower_price} - {hotel_upper_price}")
        
        ###########
        # Hostel
        ###########
        
        hostel_table = browser_soup.find("table", class_ = "ratings hostel-ratings").find_all('td')
        
        hostel_price = hostel_table[1].text.replace("\n"," ").strip()
            
        try:
            hostel_price_list = hostel_price.split(" - ")
            hostel_lower_price = hostel_price_list[0]
            hostel_upper_price = hostel_price_list[1]
        except:
            hostel_lower_price = hostel_price
            hostel_upper_price = hostel_price

        hostel_lower_priceList.append(hostel_lower_price)
        hostel_upper_priceList.append(hostel_upper_price)
        
        # get country list
        hostel_cityList.append(city_country_name)       

    except AttributeError:
        pass

    time.sleep(1)

browser.quit()

In [ ]:
###################
# put into a dict
###################

# city facts table
city_facts_dict['city_country'] = places_list
city_facts_dict['rank'] = rank_list 
city_facts_dict['daily_total_value'] = daily_total_list
city_facts_dict['population'] = population_list
city_facts_dict['metro'] = metro_list
city_facts_dict['timezone'] = timezone_list
city_facts_dict['currency'] = currency_list
city_facts_dict['airport'] = airport_list

# intro table
intro_dict['city_country'] = places_list
intro_dict['intro_para1'] = intro_p1
intro_dict['intro_para2'] = intro_p2 
intro_dict['intro_para3'] = intro_p3 
intro_dict['intro_para4'] = intro_p4 

# transportation table
transport_dict['city_country'] = transport_city_list
transport_dict['transport_mode'] = transport_mode_list
transport_dict['transport_lower_price'] = transport_lower_price_list
transport_dict['transport_upper_price'] = transport_upper_price_list

# food and drinks table
food_drinks_dict['city_country'] = food_city_list
food_drinks_dict['food_drinks_type'] = food_type_list
food_drinks_dict['food_drinks_desc'] = food_desc_list
food_drinks_dict['food_drinks_lower_price'] = food_lower_price_list
food_drinks_dict['food_drinks_upper_price'] = food_upper_price_list

# temperature and precipitation table
temp_prcp_dict['city_country'] = temp_prcp_cityList
temp_prcp_dict['month'] = month_list
temp_prcp_dict['high_temp'] = high_temp_list
temp_prcp_dict['low_temp'] = low_temp_list
temp_prcp_dict['prcp_inch'] = prcp_inch_list

# hotel pricing table
hotel_dict['city_country'] = hotel_cityList
hotel_dict['hotel_ratings'] = hotel_ratingsList
hotel_dict['hotel_lower_price'] = hotel_lower_priceList
hotel_dict['hotel_upper_price'] = hotel_upper_priceList

# hostel pricing table
hostel_dict['city_country'] = hostel_cityList
hostel_dict['hostel_lower_price'] = hostel_lower_priceList
hostel_dict['hostel_upper_price'] = hostel_upper_priceList

##################################
# put data into pandas dataframe
##################################
city_facts_df = pd.DataFrame.from_dict(city_facts_dict)
transport_df = pd.DataFrame.from_dict(transport_dict)
food_drinks_df = pd.DataFrame.from_dict(food_drinks_dict)
temp_prcp_df = pd.DataFrame.from_dict(temp_prcp_dict)
hotel_df = pd.DataFrame.from_dict(hotel_dict)
hostel_df = pd.DataFrame.from_dict(hostel_dict)
intro_df = pd.DataFrame.from_dict(intro_dict)

In [ ]:
# Connecting to SQLite
conn = sqlite3.connect('backpackers_index.db')
c = conn.cursor()

In [ ]:
###################
# City Facts
###################

# replace "-" with "" and convert into float
city_facts_df = city_facts_df.replace("-", "")
city_facts_df["daily_total_value"] = pd.to_numeric(city_facts_df["daily_total_value"])

# Create table
c.execute('CREATE TABLE CITY_FACTS (CITY_COUNTRY, RANK, DAILY_TOTAL_VALUE, POPULATION, METRO, TIMEZONE, CURRENCY, AIRPORT)')
conn.commit()
city_facts_df.to_sql('CITY_FACTS', conn, if_exists='replace', index = True)
    
# check if table in database is created

c.execute('''  
SELECT * FROM CITY_FACTS
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
###################
# Intro Paragraph
###################

# Create table
c.execute('CREATE TABLE INTRO (CITY_COUNTRY, INTRO1, INTRO2, INTRO3, INTRO4)')
conn.commit()
intro_df.to_sql('INTRO', conn, if_exists='replace', index = True)
    
# check if table in database is created

c.execute('''  
SELECT * FROM INTRO
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
###################
# Transportation
###################

# replace "-" with "" and convert into float
transport_df = transport_df.replace("-", "")
transport_df["transport_lower_price"] = pd.to_numeric(transport_df["transport_lower_price"])
transport_df["transport_upper_price"] = pd.to_numeric(transport_df["transport_upper_price"])

# Create table
c.execute('CREATE TABLE TRANSPORTATION (CITY_COUNTRY, TRANSPORT_MODE, LOWER_PRICE, UPPER_PRICE)')
conn.commit()
transport_df.to_sql('TRANSPORTATION', conn, if_exists='replace', index = True)
    
# check if table in database is created

c.execute('''  
SELECT * FROM TRANSPORTATION
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
###################
# Food and Drinks
###################

# replace "-" with "" and convert into float
food_drinks_df = food_drinks_df.replace("-", "")
food_drinks_df["food_drinks_lower_price"] = pd.to_numeric(food_drinks_df["food_drinks_lower_price"])
food_drinks_df["food_drinks_upper_price"] = pd.to_numeric(food_drinks_df["food_drinks_upper_price"])

# Create table
c.execute('CREATE TABLE FOOD_DRINKS (CITY_COUNTRY, FOOD_DRINKS_TYPE, FOOD_DRINKS_DESC, LOWER_PRICE, UPPER_PRICE)')
conn.commit()
food_drinks_df.to_sql('FOOD_DRINKS', conn, if_exists='replace', index = True)
    
# check if table in database is created

c.execute('''  
SELECT * FROM FOOD_DRINKS
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
################################
# Temperature and Precipitation
################################

# replace "-" with "" and convert into float
temp_prcp_df = temp_prcp_df.replace("-", "")
temp_prcp_df["high_temp"] = pd.to_numeric(temp_prcp_df["high_temp"])
temp_prcp_df["low_temp"] = pd.to_numeric(temp_prcp_df["low_temp"])
temp_prcp_df["prcp_inch"] = pd.to_numeric(temp_prcp_df["prcp_inch"])

# Create table
c.execute('CREATE TABLE TEMP_PRCP (CITY_COUNTRY, MONTH, HIGH_TEMP, LOW_TEMP, PRCP_INCH)')
conn.commit()
temp_prcp_df.to_sql('TEMP_PRCP', conn, if_exists='replace', index = True)
    
# check if table in database is created

c.execute('''  
SELECT * FROM TEMP_PRCP
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
#########
# Hotel
#########

# replace "-" with "" and convert into float
hotel_df = hotel_df.replace("-", "")
hotel_df["hotel_lower_price"] = pd.to_numeric(hotel_df["hotel_lower_price"])
hotel_df["hotel_upper_price"] = pd.to_numeric(hotel_df["hotel_upper_price"])

# Create table
c.execute('CREATE TABLE HOTEL (CITY_COUNTRY, RATINGS, LOWER_PRICE, UPPER_PRICE)')
conn.commit()
hotel_df.to_sql('HOTEL', conn, if_exists='replace', index = True)
    
# check if table in database is created

c.execute('''  
SELECT * FROM HOTEL
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
#########
# Hostel
#########

# replace "-" with "" and convert into float
hostel_df = hostel_df.replace("-", "")
hostel_df["hostel_lower_price"] = pd.to_numeric(hostel_df["hostel_lower_price"])
hostel_df["hostel_upper_price"] = pd.to_numeric(hostel_df["hostel_upper_price"])

# Create table
c.execute('CREATE TABLE HOSTEL (CITY_COUNTRY, LOWER_PRICE, UPPER_PRICE)')
conn.commit()
hostel_df.to_sql('HOSTEL', conn, if_exists='replace', index = True)
    
# check if table in database is created

c.execute('''  
SELECT * FROM HOSTEL
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
##################################################
# Getting the coordinates for each city
##################################################

locator = Nominatim(user_agent="myGeocoder")

top_places = {}
latitude_list = []
longitude_list = []
counter = 1

for place in places_list:   
    try:
        location = locator.geocode(place)        
        
        latitude = location.latitude 
        longitude = location.longitude
        
        print(f"{counter}. Place: {place} - Latitude: {latitude} & Longitude: {longitude}")
        counter += 1
    
    except:
        latitude = ""        
        longitude = ""

        print(f"{counter}. Place: {place} - Latitude and Longitude not found!")
        counter += 1
    
    latitude_list.append(latitude)        
    longitude_list.append(longitude)    
    
top_places['city_country'] = places_list
top_places['lat'] = latitude_list
top_places['lon'] = longitude_list

# put data into pandas dataframe
coords_df = pd.DataFrame(top_places, columns= ['city_country', 'lat', 'lon'])

# check if any city doesn't have a coord
empty_coords = coords_df[coords_df['lat']==""]
print(empty_coords)

# enter coords for San Pedro (Ambergris Caye), Belize
coords_df.loc[coords_df['city_country'] == "San Pedro (Ambergris Caye), Belize", 'lat'] = 18.0083682999
coords_df.loc[coords_df['city_country'] == "San Pedro (Ambergris Caye), Belize", 'lon'] = -87.9252862988

# check if data were correctly entered
coords_df.loc[coords_df['city_country'] == "San Pedro (Ambergris Caye), Belize"]

# Create table
c.execute('CREATE TABLE COORDS (CITY_COUNTRY, LAT, LON)')
conn.commit()
coords_df.to_sql('COORDS', conn, if_exists='replace', index = True)

# check if table in database is created

c.execute('''  
SELECT * FROM COORDS
          ''')

for row in c.fetchall():
    print(row)

In [ ]:
conn.close()